## Praca z biblioteką dash w Colabie

### Przygotowanie notatnika

Biblioteka *dash* nie jest domyślnie instalowana na maszynach udostępnianych do pracy z notatnikami Colabolatory.

Zaczynamy od zainstalowania potrzebnej biblioteki.

In [1]:
!pip install dash

Następnie importujemy biblioteki, które pozwolą nam wyświetlić wizualizację:
- *dash*, która służy do stworzenia interaktywnej wizualizacji
- *pandas*, w celu pracy z ramką danych
- *plotly.express*, aby stworzyć wykres jako część wizualizacji
- *threading*, która pozwoli nam jednocześnie uruchomić serwer i wyświetlić wizualizację.

Dodatkowo wykorzystujemy klasę *output* z biblioteki *google.colab*, która pozwala na wyświetlenie w notatniku naszej wizualizacji.

In [2]:
import dash
import pandas as pd
import plotly.express as px
import threading
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
from google.colab import output

ModuleNotFoundError: No module named 'google'

### Tworzenie wizualizacji

#### Pojedynczy blok kodu

Poniższy kod tworzy prostą wizualizację na podstawie wbudowanego zbioru danych dostępnego w *plotly.express*.

In [3]:
# define the dataset used in vis
df = px.data.tips()
# provide list of plotly theme names
plotly_theme_names = [
                      "plotly",
                      "plotly_white",
                      "plotly_dark",
                      "ggplot2",
                      "seaborn",
                      "simple_white",
                      "none"
                      ]
# define HTML font styles
font_styles={"fontFamily": "verdana", "color": "#444"}
# create app
app = dash.Dash()
# configure app layout
app.layout = html.Div(
    [
    html.H3(
        "Plotly templates review",
        style=font_styles
        ),
    dcc.Graph(id='graph'),
    dcc.Dropdown(
        id='template-dropdown',
        clearable=False,
        value="plotly",
        options=plotly_theme_names,
        style=font_styles
        ),
     ]
)
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("template-dropdown", "value")]
)
def update_figure(template):
    chart = px.scatter(
        df,
        x="total_bill",
        y="tip",
        template=template,
        color="sex",
        size="size",
        render_mode="webgl",
        title="Total bill vs tip amounts",
        labels={
            "tip": "tip amount",
            "total_bill": "total bill",
            "day": "transaction day"
        }
    )
    return chart

#### Definiowanie elementów aplikacji

Na początku przyjrzyjmy się wykorzystywanemu zbiorowi danych **Tips**, który jest częścią biblioteki *Plotly*.

In [4]:
df = px.data.tips()
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


Do stworzenia aplikacji wykorzystujemy klasę *dash.Dash()*:

In [5]:
app = dash.Dash()

Aby zbudować i wyświetlić wizualizację wykorzystujemy znaczniki języka HTML. Sam język HTML służy do budowania stron internetowych, a Dash wykorzystuje znaczniki HTML do tworzenia poszczególnych elementów dashboardu.

Poszczególne części aplikacji przechowane są w atrybucie **layout()** klasy **dash.Dash()**. 

Układ aplikacji tworzymy przez wskazanie w *layout* głównego komponentu, który zawiera wszystkie elementy wizualizacji. W tym celu wykorzystujemy **html.Div()**, który jako jedyny parametr przyjmuje listę elementów wizualizacji.

In [6]:
list_of_vis_elements = []
app.layout = html.Div(list_of_vis_elements)

Pierwszym elementem jest tytuł wizualizacji utworzony z wykorzystaniem **html.H3()**. Poza tytułem, wskazujemy tu w jaki sposób ma być sformatowany. Szczegółowo możliwości formatowania tekstu omówimy później.

In [7]:
vis_title = html.H3(
    "Plotly templates review",
    style={"fontFamily": "verdana","color": "#444"}
        )
list_of_vis_elements.append(vis_title)

Następnie wskazujemy, że kolejnym wyświetlonym elementem wizualizacji będzie wykres, którego działanie definiujemy poniżej.

In [8]:
vis_graph = dcc.Graph(id='graph')
list_of_vis_elements.append(vis_graph)

Ostatnim elementem wizualizacji jest lista rozwijalna, która pozwala użytkownikowi wybrać motyw wykorzystywany w konstrukcji wykresu.

In [9]:
theme_options = [
                 "plotly",
                 "plotly_white",
                 "plotly_dark",
                 "ggplot2",
                 "seaborn",
                 "simple_white",
                 "none"
]
vis_dropdown = dcc.Dropdown(
    id='template-dropdown',
    clearable=False,
    value="plotly",
    options = theme_options,
    style={"fontFamily": "verdana", "color": "#444"}
)
list_of_vis_elements.append(vis_dropdown)

Następny fragment kodu projektuje interakcję: wskazuje, że zmiana wartości na liście rozwijalnej jest wykorzystana do zmiany parametrów wykresu i definiuje metodę, która tworzy wykres:

In [10]:
@app.callback(
    Output('graph', 'figure'),
    [Input("template-dropdown", "value")]
)
def update_figure(template):
    chart = px.scatter(
        df,
        x="total_bill",
        y="tip",
        template=template,
        color="sex",
        size="size",
        render_mode="webgl",
        title="Total bill vs tip amounts",
        labels={
            "tip": "tip amount",
            "total_bill": "total bill",
            "day": "transaction day"
        }
    )
    return chart

### Wyświetlenie wizualizacji

Aby zbudować wizualizację z wykorzystaniem powyższego kodu, potrzebujemy:
- **uruchomić serwer**, na którym będzie dostępna - pozwala na to metoda *run_server()* klasy *dash.Dash()*,
- **wyświetlić wizualizację** dostępną na serwerze - służy do tego metoda *serve_kernel_port_as_iframe()*.

Aby wyświetlić wizualizację bez wyłączania kodu uruchamiającego serwer, wykorzystujemy bibliotekę *threading*.


In [11]:
thread = threading.Thread(target=app.run_server)
thread.start()
output.serve_kernel_port_as_iframe(port=8050, height='600')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production


NameError: name 'output' is not defined

   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Jan/2024 11:15:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2024 11:15:50] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_10_0m1685092263.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2024 11:15:50] "GET /_dash-component-suites/dash/deps/react@16.v2_10_0m1685092263.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2024 11:15:50] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_10_0m1685092263.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2024 11:15:50] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_10_0m1685092263.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2024 11:15:50] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_10_0m1685092262.min.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2024 11:15:50] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.v2_10_0m1685092263.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Jan/2024 11:15:50] "GET /_dash-compo